In [ ]:
!pip install -q RelevanceAI
!pip install sentence-splitter

In [ ]:
from sentence_splitter import SentenceSplitter, split_text_into_sentences

In [ ]:
from relevanceai import Client

In [ ]:
token = ""

In [ ]:
from relevanceai.utils import decode_workflow_token

config = decode_workflow_token(token)

In [9]:
# config={
#     'dataset_id': "walmart-store-comment-sample-csv-csv",
#     "vector_fields": ["Comments_mpnet_vector_"],
#     "text_fields" : ['Comments'],
#     "output_fields": ["one-to-many-cluster"],
#     "min_n_clusters": 50,
#     "max_n_clusters": 50,
#     "method": 'kmeans', # select from 'dbscan' or 'kmeans' or 'hybrid'
#     'authorizationToken':'',
# }

authorizationToken = config['authorizationToken']
dataset_id = config['dataset_id']
text_field = config['text_fields'][0]
new_dataset_id = config['new_dataset_id']
sentences_field = config['sentences_field']

In [7]:
splitter = SentenceSplitter(language='en')

In [ ]:
client = Client(token=authorizationToken)

ds_old = client.Dataset(dataset_id)
ds_new = client.Dataset(new_dataset_id)

In [ ]:
for docs in ds_old.chunk_dataset(select_fields=[text_field]):
    documents = []
    for doc in docs:
        text = doc[text_field] 
        if text is None:
            continue
        for sent in splitter.split(text=text):
            if sent.upper().isupper():
                document = {
                  sentences_field : sent,
                  text_field : text,
                }
                documents.append(document)
    ds_new.insert_documents(documents, create_id=True)